In [4]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os

# API KEY 정보로드
load_dotenv()

# 환경 변수 사용
langchain_project = os.getenv("LANGCHAIN_PROJECT")


In [5]:

# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith(langchain_project)

print(f"langchain_project: {langchain_project}")

LangSmith 추적을 시작합니다.
[프로젝트명]
rag_project
langchain_project: rag_project


# Document Loader

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
from langchain_community.document_loaders import pdf

In [8]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("data/gbaran.pdf")
docs = loader.load()

In [49]:
docs

[(Document(metadata={'author': 'Randall.Heldt', 'creationDate': "D:20200601003651+01'00'", 'creator': 'Microsoft® Word for Office 365', 'file_path': 'data/gbaran.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': "D:20200720090848+09'00'", 'page': 69, 'producer': 'Microsoft® Word for Office 365', 'source': 'data/gbaran.pdf', 'subject': '', 'title': 'Strategic and Tactical - Purchase Contract (Section I only)  Block 5 2017', 'total_pages': 407, 'trapped': ''}, page_content='2.3 \nWithholding \n(a) \nWhere required under APPLICABLE LAWS, COMPANY will withhold, or deduct and pay over to relevant \nAUTHORITIEs, TAXES from amounts payable to CONTRACTOR. CONTRACTOR acknowledges that any \nsum withheld or deducted will, for the purpose of the CONTRACT, be deemed to have been paid to \nCONTRACTOR and that the sum is a corresponding discharge of COMPANY´s liability to CONTRACTOR \nunder the CONTRACT. \n(b) \nWhere COMPANY makes a withholding or deduction as required by APPLICABLE LAWS, COMPA

In [9]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

In [14]:
split_documents

[Document(metadata={'source': 'data/gbaran.pdf', 'file_path': 'data/gbaran.pdf', 'page': 0, 'total_pages': 407, 'format': 'PDF 1.7', 'title': 'Strategic and Tactical - Purchase Contract (Section I only)  Block 5 2017', 'author': 'Randall.Heldt', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word for Office 365', 'producer': 'Microsoft® Word for Office 365', 'creationDate': "D:20200601003651+01'00'", 'modDate': "D:20200720090848+09'00'", 'trapped': ''}, page_content='1 \n \nINVITATION TO TENDER FOR EPC FOR GBARAN PHASE 3B - UZU WITH CPF UPGRADE- PKG 3 CPF \nUPGRADE_CW361444 \n \n1 \ntenere \n \n \n  \n \n \n The Shell Petroleum Development Company of Nigeria Limited  \nEastern Division  \nPO Box 263 Port Harcourt  \n Nigeria    \nTel  +234 (0)8070222905 Internet:www.ShellNigeria.com \n \n01 June 202031 May 2020 \nAttention:  The Managing Director.  \n \n \nDear Sirs \n \nINVITATION TO TENDER FOR EPC FOR GBARAN PHASE 3B - UZU WITH CPF UPGRADE- PKG \n3 (CPF UPGRADE) FOR SPDC \n \n

# Embedding & VectorDB Storing

In [10]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

In [16]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.

DB_PATH = "./chroma_db"
db = Chroma.from_documents(split_documents, embeddings, persist_directory=DB_PATH)


# Retrieval

https://github.com/teddylee777/langchain-kr/blob/main/10-VectorStore/04-Chroma.ipynb

In [17]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.

# 질의합니다.
query = " Withholding tax rates and application conditions"
docs = db.similarity_search(query)

# 결과를 출력합니다.
print(docs[0].page_content)

2.3 
Withholding 
(a) 
Where required under APPLICABLE LAWS, COMPANY will withhold, or deduct and pay over to relevant 
AUTHORITIEs, TAXES from amounts payable to CONTRACTOR. CONTRACTOR acknowledges that any 
sum withheld or deducted will, for the purpose of the CONTRACT, be deemed to have been paid to 
CONTRACTOR and that the sum is a corresponding discharge of COMPANY´s liability to CONTRACTOR 
under the CONTRACT. 
(b) 
Where COMPANY makes a withholding or deduction as required by APPLICABLE LAWS, COMPANY will 
provide CONTRACTOR with credit notes upon receipt from the Federal Inland Revenue Service (“FIRS”) 
or appropriate tax authority. 
(c) 
If CONTRACTOR holds a valid exemption certificate, it will provide copies or further information to 
substantiate an entitlement to avoid the withholding, which COMPANY may then rely on to apply the 
exemption. 
2.4 
Concessions, Incentives, and Exemptions


In [18]:
docs

[Document(metadata={'author': 'Randall.Heldt', 'creationDate': "D:20200601003651+01'00'", 'creator': 'Microsoft® Word for Office 365', 'file_path': 'data/gbaran.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': "D:20200720090848+09'00'", 'page': 69, 'producer': 'Microsoft® Word for Office 365', 'source': 'data/gbaran.pdf', 'subject': '', 'title': 'Strategic and Tactical - Purchase Contract (Section I only)  Block 5 2017', 'total_pages': 407, 'trapped': ''}, page_content='2.3 \nWithholding \n(a) \nWhere required under APPLICABLE LAWS, COMPANY will withhold, or deduct and pay over to relevant \nAUTHORITIEs, TAXES from amounts payable to CONTRACTOR. CONTRACTOR acknowledges that any \nsum withheld or deducted will, for the purpose of the CONTRACT, be deemed to have been paid to \nCONTRACTOR and that the sum is a corresponding discharge of COMPANY´s liability to CONTRACTOR \nunder the CONTRACT. \n(b) \nWhere COMPANY makes a withholding or deduction as required by APPLICABLE LAWS, COMPAN

In [44]:
prompt = PromptTemplate.from_template(
    """당신은 건설 프로젝트의 계약자를 위한 계약서 분석 전문가입니다. 주어진 질문에 대해 계약서의 관련 부분을 분석하고 핵심적인 답변을 제공해야 합니다. 다음 지침을 따라 답변을 작성하세요:

1. 질문의 의도를 정확히 파악하여 답변하세요.
2. 답변은 bullet point 형식으로 핵심 내용만 간결하게 작성하세요. 합니다 가 아닌 함. 이런식의 개조체 보고체로 작성하세요.
3. 제공된 계약서 조각들에서 질문에 대한 명확한 답변을 찾을 수 없다면, 어떤 부분에 대한 답변이 없는지 명시하세요.
4. 질문에 대한 답을 전혀 모르거나 관련 정보가 없다면, "주어진 정보로는 이 질문에 답변할 수 없습니다."라고 명시하세요.
5. 추측하거나 계약서에 명시되지 않은 정보를 제공하지 마세요.
6. 답변은 항상 객관적이고 사실에 기반해야 하며, 계약자의 입장에서 중요한 정보를 강조하세요.
7. 답변은 마크다운으로 작성하세요.

#질문:
{question}

#계약서 관련 부분:
{context}

#답변:"""
)

In [46]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k" : 5})

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
#    | StrOutputParser()
)

# 검색 유형을 "mmr"로 설정하여 데이터베이스를 검색기로 사용
retriever2 = db.as_retriever(search_type="mmr")

chain2 = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [47]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "How does the contract address Force Majeure events, including provisions for schedule extensions, cost allocation, duration thresholds for resolution discussions, and termination rights, and what are the specific conditions and procedures for invoking these provisions?"
response = chain.invoke(question)
print(response)

content="### Force Majeure Provisions in the Contract\n\n- **Definition of Force Majeure Events**:\n  - Includes strikes or labor disputes at a national or regional level.\n  - Government sanctions, embargoes, mandates, or laws preventing performance.\n  - Inability to timely obtain licenses, permits, or authorities' consent required for performance.\n  - Non-performance of a subcontractor affected by the above events, provided substitute performance is impracticable.\n\n- **Notification and Mitigation**:\n  - The party whose performance is delayed or prevented must:\n    - Notify the other party without delay.\n    - Use all reasonable endeavors to mitigate the effects, including acceleration of schedules upon resumption.\n    - Provide ongoing plans for resumed performance and revised schedules.\n\n- **Schedule Extensions**:\n  - The contract does not explicitly mention provisions for schedule extensions due to Force Majeure events. However, the requirement to provide revised schedul

In [33]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "How does the contract address Force Majeure events, what are the specific conditions and procedures for invoking these provisions?"
response = chain2.invoke(question)
print(response)

- Force Majeure 이벤트는 계약서에서 다음과 같은 조건을 포함함:
  - 전국적 또는 지역적 수준의 파업 또는 노동 분쟁, 계약자 그룹 또는 회사 그룹에 속하지 않는 노동을 포함하여 계약 이행에 중대한 영향을 미치는 경우.
  - 정부 제재, 금수 조치, 명령 또는 법률로 인해 이행이 불가능한 경우.
  - 계약서에 명시된 경우를 제외하고, 이행에 필요한 라이선스, 허가 또는 당국의 동의를 적시에 얻지 못한 경우.
  - 하도급업체가 위의 Force Majeure 이벤트 중 하나에 영향을 받아 이행하지 못하는 경우. 단, 다른 하도급업체에 의한 대체 이행이 실질적으로 불가능한 경우에만 이행이 면제됨.

- Force Majeure 조항을 발동하기 위한 절차:
  - 이행이 지연되거나 방해받는 당사자는 지체 없이 다른 당사자에게 통지해야 함.
  - 이행 재개 시 일정 가속화 등 합리적인 노력을 기울여 영향을 완화해야 함.
  - 이행 재개 계획과 수정된 일정을 지속적으로 제공해야 함.

- Force Majeure 이벤트로 인해 90일 연속 또는 180일 누적 지연이 발생할 경우, 회사는 계약 또는 일부 범위를 종료할 수 있음. 단, 회사가 변동 명령을 제공하는 경우는 예외임.

- 제공된 정보에서 이 질문에 대한 명확한 답변을 찾을 수 없습니다.


In [35]:
retriever2.get_relevant_documents(query)


e:\Dev\Flask_RAG\flask_env\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(metadata={'author': 'Randall.Heldt', 'creationDate': "D:20200601003651+01'00'", 'creator': 'Microsoft® Word for Office 365', 'file_path': 'data/gbaran.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': "D:20200720090848+09'00'", 'page': 69, 'producer': 'Microsoft® Word for Office 365', 'source': 'data/gbaran.pdf', 'subject': '', 'title': 'Strategic and Tactical - Purchase Contract (Section I only)  Block 5 2017', 'total_pages': 407, 'trapped': ''}, page_content='2.3 \nWithholding \n(a) \nWhere required under APPLICABLE LAWS, COMPANY will withhold, or deduct and pay over to relevant \nAUTHORITIEs, TAXES from amounts payable to CONTRACTOR. CONTRACTOR acknowledges that any \nsum withheld or deducted will, for the purpose of the CONTRACT, be deemed to have been paid to \nCONTRACTOR and that the sum is a corresponding discharge of COMPANY´s liability to CONTRACTOR \nunder the CONTRACT. \n(b) \nWhere COMPANY makes a withholding or deduction as required by APPLICABLE LAWS, COMPAN

In [36]:
docs = db.similarity_search_with_score(query)
docs

[(Document(metadata={'author': 'Randall.Heldt', 'creationDate': "D:20200601003651+01'00'", 'creator': 'Microsoft® Word for Office 365', 'file_path': 'data/gbaran.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': "D:20200720090848+09'00'", 'page': 69, 'producer': 'Microsoft® Word for Office 365', 'source': 'data/gbaran.pdf', 'subject': '', 'title': 'Strategic and Tactical - Purchase Contract (Section I only)  Block 5 2017', 'total_pages': 407, 'trapped': ''}, page_content='2.3 \nWithholding \n(a) \nWhere required under APPLICABLE LAWS, COMPANY will withhold, or deduct and pay over to relevant \nAUTHORITIEs, TAXES from amounts payable to CONTRACTOR. CONTRACTOR acknowledges that any \nsum withheld or deducted will, for the purpose of the CONTRACT, be deemed to have been paid to \nCONTRACTOR and that the sum is a corresponding discharge of COMPANY´s liability to CONTRACTOR \nunder the CONTRACT. \n(b) \nWhere COMPANY makes a withholding or deduction as required by APPLICABLE LAWS, COMPA

### 혹은

여기서부터는 체인에 대한 이해가 필요 함. 
하나의 파일로 만들어서 여러가지 테스트가 필요 함.

In [43]:
relevant_docs = retriever.get_relevant_documents("What are the customs procedures, responsibilities, and financial implications for project-related materials and equipment, including clearance duties, exemptions, cost bearings, and provisions for regulatory changes?")

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

question = "How does the contract address Force Majeure events, what are the specific conditions and procedures for invoking these provisions?"
response = chain2.invoke(question)
print(response)

print("===" * 20)
print(f"문서의 개수: {len(relevant_docs)}")
print("[검색 결과]\n")
for i in range(len(relevant_docs)):
    print(relevant_docs[i].page_content)
    print("===" * 20)

- Force Majeure 이벤트는 다음과 같은 조건을 포함함:
  - 전국적 또는 지역적 수준의 파업 또는 노동 분쟁, 계약자의 그룹이나 회사의 그룹에 속하지 않는 노동을 포함하여 계약 이행에 중대한 영향을 미치는 경우.
  - 정부 제재, 금수 조치, 명령 또는 법률로 인해 이행이 불가능한 경우.
  - 계약서에 명시된 경우를 제외하고, 이행에 필요한 라이선스, 허가 또는 당국의 동의를 적시에 얻지 못한 경우.
  - 하도급업체가 위의 Force Majeure 이벤트 중 하나에 영향을 받은 경우. 단, 다른 하도급업체에 의한 대체 이행이 실질적으로 불가능한 경우에만 이행이 면제됨.

- Force Majeure 조항을 발동하기 위한 절차:
  - 이행이 지연되거나 방해받는 당사자는 지체 없이 다른 당사자에게 통지해야 함.
  - 이행이 재개될 때까지 모든 합리적인 노력을 기울여 영향을 완화해야 함.
  - 지속적으로 이행 재개 계획과 수정된 일정을 제공해야 함.

- Force Majeure 이벤트로 인해 90일 연속 또는 180일 누적 지연이 발생할 경우, 회사는 계약 또는 일부 범위를 종료할 수 있음. 단, 회사가 Variation Order를 제공하는 경우는 예외임.

- 제공된 정보에서 이 질문에 대한 명확한 답변을 찾을 수 없습니다.
문서의 개수: 5
[검색 결과]

preservation and issue for construction. 
CONTRACTOR will provide and use materials management/tracking solutions, tools, software or 
applications for efficient project execution.  
CONTRACTOR shall be responsible for organising, coordinating and performing all shipping and 
transportation arrangements and activities for MATER

# Thoughts

## 메타데이터를 추가하는 방법

In [22]:
project_name = "gbaran infill"
country = "Nigeria"

documents_with_metadata = []
for doc in split_documents:
    doc.metadata["project_name"] = project_name
    doc.metadata["country"] = country
    documents_with_metadata.append(doc)
    
documents_with_metadata[5]

Document(metadata={'source': 'data/gbaran.pdf', 'file_path': 'data/gbaran.pdf', 'page': 2, 'total_pages': 407, 'format': 'PDF 1.7', 'title': 'Strategic and Tactical - Purchase Contract (Section I only)  Block 5 2017', 'author': 'Randall.Heldt', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word for Office 365', 'producer': 'Microsoft® Word for Office 365', 'creationDate': "D:20200601003651+01'00'", 'modDate': "D:20200720090848+09'00'", 'trapped': '', 'project_name': 'gbaran infill', 'country': 'Nigeria'}, page_content='3 \n \nINVITATION TO TENDER FOR EPC FOR GBARAN PHASE 3B - UZU WITH CPF UPGRADE- PKG 3 CPF \nUPGRADE_CW361444 \n \n3 \n \n \n \nTENDER ACKNOWLEDGEMENT FORM \n \nThe Shell Petroleum Development Company of Nigeria Limited \nFor the Attention of Ojonimi Osayi.  \n \nRE: ACKNOWLEDGEMENT FOR EPC FOR GBARAN PHASE 3B - UZU WITH CPF UPGRADE- PKG 3 \n(CPF UPGRADE) TENDER No. CW361444 \n \n(a) \nWe acknowledge receipt of the ITT and we agree to maintain confidentiality in r

## 메타데이터 기준 필터링

https://github.com/teddylee777/langchain-kr/blob/main/10-VectorStore/04-Chroma.ipynb


In [3]:
from prompts.prompt_manager import PromptManager

# PromptManager 초기화 (경로는 상대 경로 또는 절대 경로 사용)
prompt_manager = PromptManager('prompts/yaml/retrieval_prompts.yaml')

# 프롬프트 가져오기
retrieval_system_prompt = prompt_manager.get_prompt('retrieval', 'system')
retrieval_user_prompt = prompt_manager.get_prompt('retrieval', 'user', query="What is RAG?", context="RAG stands for...")
generation_system_prompt = prompt_manager.get_prompt('generation', 'system')

print(retrieval_system_prompt)
print(retrieval_user_prompt)
print(generation_system_prompt)

You are a retrieval assistant. Your task is to analyze the given query and context to find the most relevant information.

Query: What is RAG?
Context: RAG stands for...
Find the most relevant information from the context to answer the query.

You are an AI assistant specialized in generating responses based on given context and query.

